In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
import pywt
import xlwt,xlrd,xlsxwriter
from sklearn import preprocessing
import pywt.data
import xlutils.copy
import math
import pandas as pd

#  逻辑复制


In [2]:

pi=np.pi

cos=np.cos
sin=np.sin
exp=np.exp




## Gabor

In [3]:


def build_filters():
     filters = []
     ksize = 21 
     lamda = np.pi/2.0         # 波长
     thetas=[0,pi/8,pi/4,pi*3/8,pi/2,pi*5/8,pi*3/4,pi*7/8]
     for theta in thetas: #gabor方向，0°，45°，90°，135°，共四个
             kern = cv2.getGaborKernel((ksize, ksize), 1.0, theta, lamda, 0.5, 0, ktype=cv2.CV_32F)
             filters.append(kern)
    
     return filters
    
def process(img, filters,tz):
    accum = np.zeros_like(img)
    for kern in filters:
         fimg = cv2.filter2D(img, cv2.CV_8UC1,kern)
         fimg_abs=list(map(abs,fimg))
         fimg_abs=np.array(fimg_abs)
         
         fimg_max=np.max(fimg)
         m=list(fimg_abs/fimg_max)
         
         fimg_std=np.std(m)
         fimg_mean=np.mean(m)
         tz.append(fimg_mean)
         tz.append(fimg_std)



## 小波提取

In [4]:
# ca4,cg4,cg3,cg2,cg1=pywt.wavedec2(img,w,mode="symmetric", level=4, axes=(-2, -1))##CA,(CH,CV,CD)
# cg4=np.array(cg4)
# c=ca4+cg4[0]


def get_a(coeffs,w):
    a=[]
    coeffs_r=coeffs.copy()

    coeffs_r[1]=np.zeros_like(coeffs[1])
    coeffs_r[2] = np.zeros_like(coeffs[2])
    coeffs_r[3] = np.zeros_like(coeffs[3])
    coeffs_r[4] = np.zeros_like(coeffs[4])

    m = pywt.waverec2(coeffs_r, w) 
    
    a_mean=np.mean(np.abs(m))
    a_std=np.std(m)
    
    a.append(a_mean)
    a.append(a_std)
    
    return a

def get_h(coeffs,i,w):
    coeffs_r=coeffs.copy()

    coeffs_r[0]=np.zeros_like(coeffs[0])
    if(i==1):
        coeffs_r[2] = np.zeros_like(coeffs[2])
        coeffs_r[3] = np.zeros_like(coeffs[3])
        coeffs_r[4] = np.zeros_like(coeffs[4])
        
        coeffs_r[1]=list(coeffs_r[1])
        coeffs_r[1][1]=np.zeros_like(coeffs_r[1][1])
        coeffs_r[1][2]=np.zeros_like(coeffs_r[1][2])
    if(i==2):
        coeffs_r[1] = np.zeros_like(coeffs[1])
        coeffs_r[3] = np.zeros_like(coeffs[3])
        coeffs_r[4] = np.zeros_like(coeffs[4])
        
        coeffs_r[2]=list(coeffs_r[2])
        coeffs_r[2][1]=np.zeros_like(coeffs_r[2][1])
        coeffs_r[2][2]=np.zeros_like(coeffs_r[2][2])
        
    if(i==3):
        coeffs_r[1] = np.zeros_like(coeffs[1])
        coeffs_r[2] = np.zeros_like(coeffs[2])
        coeffs_r[4] = np.zeros_like(coeffs[4])
        
        coeffs_r[3]=list(coeffs_r[3])
        coeffs_r[3][1]=np.zeros_like(coeffs_r[3][1])
        coeffs_r[3][2]=np.zeros_like(coeffs_r[3][2])
    if(i==4):
        coeffs_r[1] = np.zeros_like(coeffs[1])
        coeffs_r[3] = np.zeros_like(coeffs[3])
        coeffs_r[2] = np.zeros_like(coeffs[2])
        
        coeffs_r[4]=list(coeffs_r[4])
        coeffs_r[4][1]=np.zeros_like(coeffs_r[4][1])
        coeffs_r[4][2]=np.zeros_like(coeffs_r[4][2])

   

    h=[]
    m = pywt.waverec2(coeffs_r, w) 

    h_mean=np.mean(np.abs(m))
    h_std=np.std(m)

    h.append(h_mean)
    h.append(h_std)
    
    return h


def get_v(coeffs,i,w):
    coeffs_r=coeffs.copy()

    coeffs_r[0]=np.zeros_like(coeffs[0])
    if(i==1):
        coeffs_r[2] = np.zeros_like(coeffs[2])
        coeffs_r[3] = np.zeros_like(coeffs[3])
        coeffs_r[4] = np.zeros_like(coeffs[4])
        
        coeffs_r[1]=list(coeffs_r[1])
        coeffs_r[1][0]=np.zeros_like(coeffs_r[1][0])
        coeffs_r[1][2]=np.zeros_like(coeffs_r[1][2])
    if(i==2):
        coeffs_r[1] = np.zeros_like(coeffs[1])
        coeffs_r[3] = np.zeros_like(coeffs[3])
        coeffs_r[4] = np.zeros_like(coeffs[4])
        
        coeffs_r[2]=list(coeffs_r[2])
        coeffs_r[2][0]=np.zeros_like(coeffs_r[2][0])
        coeffs_r[2][2]=np.zeros_like(coeffs_r[2][2])
        
    if(i==3):
        coeffs_r[1] = np.zeros_like(coeffs[1])
        coeffs_r[2] = np.zeros_like(coeffs[2])
        coeffs_r[4] = np.zeros_like(coeffs[4])
        
        coeffs_r[3]=list(coeffs_r[3])
        coeffs_r[3][0]=np.zeros_like(coeffs_r[3][0])
        coeffs_r[3][2]=np.zeros_like(coeffs_r[3][2])
    if(i==4):
        coeffs_r[1] = np.zeros_like(coeffs[1])
        coeffs_r[3] = np.zeros_like(coeffs[3])
        coeffs_r[2] = np.zeros_like(coeffs[2])
        
        coeffs_r[4]=list(coeffs_r[4])
        coeffs_r[4][0]=np.zeros_like(coeffs_r[4][0])
        coeffs_r[4][2]=np.zeros_like(coeffs_r[4][2])

   

    v=[]
    m = pywt.waverec2(coeffs_r, w) 

    v_mean=np.mean(np.abs(m))
    v_std=np.std(m)

    v.append(v_mean)
    v.append(v_std)
    
    return v


def get_d(coeffs,i,w):
    coeffs_r=coeffs.copy()

    coeffs_r[0]=np.zeros_like(coeffs[0])
    if(i==1):
        coeffs_r[2] = np.zeros_like(coeffs[2])
        coeffs_r[3] = np.zeros_like(coeffs[3])
        coeffs_r[4] = np.zeros_like(coeffs[4])
        
        coeffs_r[1]=list(coeffs_r[1])
        coeffs_r[1][0]=np.zeros_like(coeffs_r[1][0])
        coeffs_r[1][1]=np.zeros_like(coeffs_r[1][1])
    if(i==2):
        coeffs_r[1] = np.zeros_like(coeffs[1])
        coeffs_r[3] = np.zeros_like(coeffs[3])
        coeffs_r[4] = np.zeros_like(coeffs[4])
        
        coeffs_r[2]=list(coeffs_r[2])
        coeffs_r[2][0]=np.zeros_like(coeffs_r[2][0])
        coeffs_r[2][1]=np.zeros_like(coeffs_r[2][1])
        
    if(i==3):
        coeffs_r[1] = np.zeros_like(coeffs[1])
        coeffs_r[2] = np.zeros_like(coeffs[2])
        coeffs_r[4] = np.zeros_like(coeffs[4])
        
        coeffs_r[3]=list(coeffs_r[3])
        coeffs_r[3][0]=np.zeros_like(coeffs_r[3][0])
        coeffs_r[3][1]=np.zeros_like(coeffs_r[3][1])
    if(i==4):
        coeffs_r[1] = np.zeros_like(coeffs[1])
        coeffs_r[3] = np.zeros_like(coeffs[3])
        coeffs_r[2] = np.zeros_like(coeffs[2])
        
        coeffs_r[4]=list(coeffs_r[4])
        coeffs_r[4][0]=np.zeros_like(coeffs_r[4][0])
        coeffs_r[4][1]=np.zeros_like(coeffs_r[4][1])

    d=[]
    m = pywt.waverec2(coeffs_r, w) 

    d_mean=np.mean(np.abs(m))
    d_std=np.std(m)

    d.append(d_mean)
    d.append(d_std)
    
    
    return d



In [5]:
def wavelet_transform(img):
    w = pywt.Wavelet("db4")
    coeffs=pywt.wavedec2(img,w,mode="symmetric", level=4, axes=(-2, -1))
    
    value_total=[]
    for i in range(1,5):
    
                a=get_a(coeffs,w)
                h=get_h(coeffs,i,w)
                v=get_v(coeffs,i,w)
                d=get_d(coeffs,i,w)

                value=[]
                value.extend(a)
                value.extend(h)
                value.extend(v)
                value.extend(d)

                value_total.extend(value)

    return value_total

## 灰度共生

In [6]:
gray_level = 16

def maxGrayLevel(img):
    max_gray_level=0
    (height,width)=img.shape
#     print (height,width)
    for y in range(height):
        for x in range(width):
            if img[y][x] > max_gray_level:
                max_gray_level = img[y][x]
    return max_gray_level+1


def getGlcm(input,d_x,d_y):
    srcdata=input.copy()
    ret=[[0.0 for i in range(gray_level)] for j in range(gray_level)]
    (height,width) = input.shape
    
    max_gray_level=maxGrayLevel(input)
    
    if max_gray_level > gray_level:
        for j in range(height):
            for i in range(width):
                srcdata[j][i] = srcdata[j][i]*gray_level / max_gray_level
                
    
    for j in range(height-d_y):
        for i in range(width-d_x):
              rows = srcdata[j][i]
              cols = srcdata[j + d_y][i+d_x]
              ret[rows][cols]+=1.0
                
                
    for i in range(gray_level):
        for j in range(gray_level):
            ret[i][j]/=float(height*width)

    return ret   


def feature_computer(p):
    Con=0.0
    Eng=0.0
    Asm=0.0
    Idm=0.0
    for i in range(gray_level):
        for j in range(gray_level):
            Con+=(i-j)*(i-j)*p[i][j]
            Asm+=p[i][j]*p[i][j]
            Idm+=p[i][j]/(1+(i-j)*(i-j))
            if p[i][j]>0.0:
                Eng+=p[i][j]*math.log(p[i][j])
                
    return Asm,Con,-Eng,Idm           



def test(img):
    value=[]
    try:
        img_shape=img.shape
    except:
        print ('imread error')
        return
    
    img=cv2.resize(img,(int(img_shape[1]/2),int(img_shape[0]/2)),interpolation=cv2.INTER_CUBIC)

    
    
    glcm_0=getGlcm(img, 1,0)
    
    asm,con,eng,idm=feature_computer(glcm_0)
    
    return [asm,con,eng,idm]

# 测试提取

## 特征提取

In [7]:
def fetch_features(path):   ###特征提取   
        files = os.listdir(path)
        files_num=len(files)
        tz_total=[]
        for i in range(files_num):
            tz=[]

            img=cv2.imread(path+"/"+files[i])
            name=files[i]

            img=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
            
            print("正在处理{}".format(name))

            img_average=np.mean(img)
            img_std=np.std(img)
            img_values=[img_average,img_std]

            tz.extend(img_values)

            ##Gabor
            t_gabor=[]
            filters=build_filters()
            process(img,filters,t_gabor)
            tz.extend(t_gabor)
            
            print("{}已经过Gabor滤波！".format(name))

            ##小波

            t_xb=[]
            t_xb=wavelet_transform(img)
            tz.extend(t_xb)
            print("{}已经过小波处理！".format(name))

            ##灰度共生
            g_v=test(img)
            tz.append(g_v[1])
            tz.append(g_v[2])
            tz.append(g_v[3])
            print("{}已得到灰度共生矩阵！".format(name))


            tz_total.append(tz)
            print("{}的特征已得到！".format(name))
            
            
            
        print("特征获取结束")
        return tz_total

## 特征写入

In [8]:
def data_write(save_path, datas,file_path,tp):##特征写入
    
        files = os.listdir(file_path)

        f = xlwt.Workbook()
        sheet1 = f.add_sheet(u'sheet1',cell_overwrite_ok=True)

        for j in range(len(datas[0])+1):
            if (j==0):
                sheet1.write(0,j,"文件名")
            else:
                sheet1.write(0,j,"特征{}".format(j))

        for j in range(len(datas)):
                sheet1.write(j+1,0,"{}".format(files[j]))  


        sheet1.write(0,(len(datas[0])+1),"分类")




        i=1
        for data in datas:
            for j in range(len(data)):

                    sheet1.write(i,j+1,data[j])
                    sheet1.write(i,len(data)+1,tp)
            i = i + 1

        if tp==1:
            f.save(save_path+"/1.xls")
        if tp==2:           
            f.save(save_path+"/2.xls")
        if tp==3:           
            f.save(save_path+"/3.xls")
        if tp==0:
            f.save(save_path+"/0.xls")     
    ####################################
    
    


## 获取特征

In [9]:
def get_feature_3():###3种标签
    save_path=input("请输入保存的地址")
    for i in range(1,4):
        file_path=input("请输入{}的地址".format(i))
        
        datas=fetch_features(file_path)
        data_write(save_path,datas,file_path,i)
        print("{}已完成！".format(i))
    print("开始合并文件...")
    dfs=[]
    for fn in ("1.xls","2.xls","3.xls"):
        dfs.append(pd.read_excel(fn))

    df=pd.concat(dfs)
    df.to_excel("result.xls",index=False)
    print("已经全部完成")

In [10]:
def get_feature_2():###2种标签
    save_path=input("请输入保存的地址")
    for i in range(0,2):
        file_path=input("请输入{}的地址".format(i))
        
        datas=fetch_features(file_path)
        data_write(save_path,datas,file_path,i)
        print("{}已完成！".format(i))
    print("开始合并文件...")
    dfs=[]
    for fn in ("0.xls","1.xls"):
        dfs.append(pd.read_excel(fn))

    df=pd.concat(dfs)
    df.to_excel("result_01.xls",index=False)
    print("已经全部完成")

# 测试

In [11]:
get_feature_2()

请输入保存的地址C:/Users/lenovo/Desktop/branch
请输入0的地址C:/Users/lenovo/Desktop/branch/A
正在处理01-LC-ser.bmp
01-LC-ser.bmp已经过Gabor滤波！
01-LC-ser.bmp已经过小波处理！
01-LC-ser.bmp已得到灰度共生矩阵！
01-LC-ser.bmp的特征已得到！
正在处理01-LM-ser.bmp
01-LM-ser.bmp已经过Gabor滤波！
01-LM-ser.bmp已经过小波处理！
01-LM-ser.bmp已得到灰度共生矩阵！
01-LM-ser.bmp的特征已得到！
正在处理02-LC-ser.bmp
02-LC-ser.bmp已经过Gabor滤波！
02-LC-ser.bmp已经过小波处理！
02-LC-ser.bmp已得到灰度共生矩阵！
02-LC-ser.bmp的特征已得到！
正在处理02-LM-ser.bmp
02-LM-ser.bmp已经过Gabor滤波！
02-LM-ser.bmp已经过小波处理！
02-LM-ser.bmp已得到灰度共生矩阵！
02-LM-ser.bmp的特征已得到！
正在处理02-RC-ser.bmp


E:\Anaconda\envs\ml\lib\site-packages\pywt\_multilevel.py:45: UserWarning: Level value of 4 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))


02-RC-ser.bmp已经过Gabor滤波！
02-RC-ser.bmp已经过小波处理！
02-RC-ser.bmp已得到灰度共生矩阵！
02-RC-ser.bmp的特征已得到！
正在处理02-RM-ser.bmp
02-RM-ser.bmp已经过Gabor滤波！
02-RM-ser.bmp已经过小波处理！
02-RM-ser.bmp已得到灰度共生矩阵！
02-RM-ser.bmp的特征已得到！
正在处理03-LC-ser.bmp
03-LC-ser.bmp已经过Gabor滤波！
03-LC-ser.bmp已经过小波处理！
03-LC-ser.bmp已得到灰度共生矩阵！
03-LC-ser.bmp的特征已得到！
正在处理03-LM-ser.bmp
03-LM-ser.bmp已经过Gabor滤波！
03-LM-ser.bmp已经过小波处理！
03-LM-ser.bmp已得到灰度共生矩阵！
03-LM-ser.bmp的特征已得到！
正在处理04-RC-ser.bmp
04-RC-ser.bmp已经过Gabor滤波！
04-RC-ser.bmp已经过小波处理！
04-RC-ser.bmp已得到灰度共生矩阵！
04-RC-ser.bmp的特征已得到！
正在处理04-RM-ser.bmp
04-RM-ser.bmp已经过Gabor滤波！
04-RM-ser.bmp已经过小波处理！
04-RM-ser.bmp已得到灰度共生矩阵！
04-RM-ser.bmp的特征已得到！
正在处理05-RC-ser.bmp
05-RC-ser.bmp已经过Gabor滤波！
05-RC-ser.bmp已经过小波处理！
05-RC-ser.bmp已得到灰度共生矩阵！
05-RC-ser.bmp的特征已得到！
正在处理05-RM-ser.bmp
05-RM-ser.bmp已经过Gabor滤波！
05-RM-ser.bmp已经过小波处理！
05-RM-ser.bmp已得到灰度共生矩阵！
05-RM-ser.bmp的特征已得到！
正在处理06-LC-ser.bmp
06-LC-ser.bmp已经过Gabor滤波！
06-LC-ser.bmp已经过小波处理！
06-LC-ser.bmp已得到灰度共生矩阵！
06-LC-ser.bmp的特征已得到！
正在处理06-LM-ser.bmp
06-LM-ser.

35-LC-ser.bmp已经过Gabor滤波！
35-LC-ser.bmp已经过小波处理！
35-LC-ser.bmp已得到灰度共生矩阵！
35-LC-ser.bmp的特征已得到！
正在处理35-LM-ser.bmp
35-LM-ser.bmp已经过Gabor滤波！
35-LM-ser.bmp已经过小波处理！
35-LM-ser.bmp已得到灰度共生矩阵！
35-LM-ser.bmp的特征已得到！
正在处理36-LC-ser.bmp
36-LC-ser.bmp已经过Gabor滤波！
36-LC-ser.bmp已经过小波处理！
36-LC-ser.bmp已得到灰度共生矩阵！
36-LC-ser.bmp的特征已得到！
正在处理36-LM-ser.bmp
36-LM-ser.bmp已经过Gabor滤波！
36-LM-ser.bmp已经过小波处理！
36-LM-ser.bmp已得到灰度共生矩阵！
36-LM-ser.bmp的特征已得到！
正在处理37-LC-ser.bmp
37-LC-ser.bmp已经过Gabor滤波！
37-LC-ser.bmp已经过小波处理！
37-LC-ser.bmp已得到灰度共生矩阵！
37-LC-ser.bmp的特征已得到！
正在处理37-LM-ser.bmp
37-LM-ser.bmp已经过Gabor滤波！
37-LM-ser.bmp已经过小波处理！
37-LM-ser.bmp已得到灰度共生矩阵！
37-LM-ser.bmp的特征已得到！
特征获取结束
0已完成！
请输入1的地址C:/Users/lenovo/Desktop/branch/B
正在处理39-LC-ser.bmp
39-LC-ser.bmp已经过Gabor滤波！
39-LC-ser.bmp已经过小波处理！
39-LC-ser.bmp已得到灰度共生矩阵！
39-LC-ser.bmp的特征已得到！
正在处理39-LM-ser.bmp
39-LM-ser.bmp已经过Gabor滤波！
39-LM-ser.bmp已经过小波处理！
39-LM-ser.bmp已得到灰度共生矩阵！
39-LM-ser.bmp的特征已得到！
正在处理40-RC-ser.bmp
40-RC-ser.bmp已经过Gabor滤波！
40-RC-ser.bmp已经过小波处理！
40-RC-ser.bmp已得到灰度共生

60-RC-ser-4.bmp已经过小波处理！
60-RC-ser-4.bmp已得到灰度共生矩阵！
60-RC-ser-4.bmp的特征已得到！
正在处理60-RC-ser.bmp
60-RC-ser.bmp已经过Gabor滤波！
60-RC-ser.bmp已经过小波处理！
60-RC-ser.bmp已得到灰度共生矩阵！
60-RC-ser.bmp的特征已得到！
正在处理60-RM-ser-1.bmp
60-RM-ser-1.bmp已经过Gabor滤波！
60-RM-ser-1.bmp已经过小波处理！
60-RM-ser-1.bmp已得到灰度共生矩阵！
60-RM-ser-1.bmp的特征已得到！
正在处理60-RM-ser-2.bmp
60-RM-ser-2.bmp已经过Gabor滤波！
60-RM-ser-2.bmp已经过小波处理！
60-RM-ser-2.bmp已得到灰度共生矩阵！
60-RM-ser-2.bmp的特征已得到！
正在处理60-RM-ser-3.bmp
60-RM-ser-3.bmp已经过Gabor滤波！
60-RM-ser-3.bmp已经过小波处理！
60-RM-ser-3.bmp已得到灰度共生矩阵！
60-RM-ser-3.bmp的特征已得到！
正在处理60-RM-ser.bmp
60-RM-ser.bmp已经过Gabor滤波！
60-RM-ser.bmp已经过小波处理！
60-RM-ser.bmp已得到灰度共生矩阵！
60-RM-ser.bmp的特征已得到！
正在处理61-RC-ser.bmp
61-RC-ser.bmp已经过Gabor滤波！
61-RC-ser.bmp已经过小波处理！
61-RC-ser.bmp已得到灰度共生矩阵！
61-RC-ser.bmp的特征已得到！
正在处理61-RM-ser.bmp
61-RM-ser.bmp已经过Gabor滤波！
61-RM-ser.bmp已经过小波处理！
61-RM-ser.bmp已得到灰度共生矩阵！
61-RM-ser.bmp的特征已得到！
正在处理62-RC-ser.bmp
62-RC-ser.bmp已经过Gabor滤波！
62-RC-ser.bmp已经过小波处理！
62-RC-ser.bmp已得到灰度共生矩阵！
62-RC-ser.bmp的特征已得到！
正在处理62-RM-ser.bmp